In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering
from sklearn import metrics

from sqlalchemy import create_engine
import datetime 
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import matplotlib.cm as cm
import matplotlib
from TurbineTimeSeries.storage import MachineDataStore
from TurbineTimeSeries.transformations import StepSize, PowerStepSize

In [2]:
model_number = 2

store = MachineDataStore('.config')

# model_data_hr = (store.query(model_number,'1hr')
#                  .not_null(['timestamp','psn'])
# #                  .exclude_psn([44,52,54,70])
#                  .execute())

model_data_min = (store.query(model_number,'10min')
                  .not_null(['timestamp','psn'])
                  .exclude_psn([44,52,54,70])
                  .execute())

model_data_min.head()

/Users/jillianjarrett/anaconda2/envs/py36envir/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


SELECT * FROM sensor_readings_model2_10min WHERE timestamp IS NOT NULL AND psn IS NOT NULL AND psn not in (44,52,54,70)


,id,lo_c_dp1,f_c_dp1,f_c_dp2,f_c_dp5,pe_c_dt1,g_c_dt1,g_c_dt2,lo_c_dt5,c_dt5_1,...,v_d_2b,lo_t5,g_t2,f_t1,sum_enr,g_t3,g_pct1,f_pos1,timestamp,psn
0,748834,0.266275,0.069489,0.028374,0.270104,0.257778,0.007999,0.165999,0.424705,-0.200000,...,0.194285,0.760000,0.840000,0.3595,86.873418,0.4276,0.34244,0.482371,2017-07-03 03:19:59.999998,62
1,748835,0.265987,0.083112,0.011345,0.269696,0.252222,0.012002,0.142002,0.424707,-0.187334,...,0.197142,0.759565,0.832572,0.3580,86.907342,0.4336,0.35189,0.483574,2017-07-03 03:30:00.000003,62
2,748836,0.263590,0.096204,-0.005676,0.271058,0.251112,0.016002,0.154001,0.421177,-0.201833,...,0.194285,0.758261,0.836000,0.3585,86.941013,0.4300,0.36265,0.483574,2017-07-03 03:39:59.999997,62
3,748837,0.262392,0.109827,-0.005676,0.270785,0.248890,0.016002,0.151999,0.424707,-0.193834,...,0.188571,0.759565,0.835428,0.3600,86.974177,0.4308,0.33308,0.483574,2017-07-03 03:50:00.000002,62
4,748838,0.263245,0.105286,0.011345,0.270785,0.255554,0.011999,0.150001,0.425882,-0.194666,...,0.185714,0.761304,0.836572,0.3600,87.007848,0.4300,0.34888,0.483574,2017-07-03 03:59:59.999996,62


In [3]:
freq = '10min'
model_data = model_data_min
len(model_data)

1602326

In [80]:
skipped_cols = ['sum_esn','sum_eng_st', 'sum_eng_h']
index_cols = ['id','timestamp','psn']
data_cols = [c for c in model_data.columns if (c not in index_cols) and (c not in skipped_cols)]

In [81]:
missing_values = model_data.isnull().sum().sort_values()
sparse_cols = [x for x in missing_values.index if missing_values[x] > 30000]
clean_data_cols = [x for x in data_cols if x not in sparse_cols]

In [164]:
data = model_data[index_cols + clean_data_cols].dropna().reset_index()
clean_data = StandardScaler().fit_transform(data[clean_data_cols])

pca =  PCA().fit(clean_data)
reduced = pca.transform(clean_data)

In [165]:
reduced_df = pd.DataFrame(reduced)
reduced_df['psn'] = data.psn.values
reduced_df['timestamp'] = data.timestamp.values
eig_df = reduced_df[['psn','timestamp',0,1,2,3,4,5,6,7,8,9,10]]
eig_df.columns = ['psn','timestamp','eig0','eig1','eig2','eig3','eig4','eig5','eig6','eig7','eig8','eig9','eig10']
eig_df.head()

,psn,timestamp,eig0,eig1,eig2,eig3,eig4,eig5,eig6,eig7,eig8,eig9,eig10
0,62,2017-07-03 03:19:59.999998,10.874386,-4.582596,0.640216,-0.011155,-1.099033,0.323496,-2.707881,-2.256011,1.606701,0.093378,-0.298582
1,62,2017-07-03 03:30:00.000003,10.566446,-4.466257,0.471729,-0.082046,-1.311584,-0.124291,-3.036025,-1.830366,1.494335,0.074276,-0.143505
2,62,2017-07-03 03:39:59.999997,10.781455,-4.549097,0.448211,-0.131743,-1.230701,-0.283143,-3.139220,-2.062612,1.515007,0.187542,-0.323860
3,62,2017-07-03 03:50:00.000002,10.577235,-4.609505,0.567070,-0.160696,-1.178241,-0.283500,-2.826825,-1.793481,1.402721,0.192670,-0.098927
4,62,2017-07-03 03:59:59.999996,10.646234,-4.785537,0.351168,-0.387728,-1.198526,-0.479855,-3.468351,-1.612749,1.391862,0.013799,-0.015457


In [166]:
#uses jareds stepsize function
temp_df = eig_df[['timestamp','psn','eig0']].reset_index()
temp_df.set_index(['psn', 'timestamp'], inplace=True)
del temp_df['index']
eig_ss_class = StepSize(threshold=5,rolling_days=7)

transformed_eig_ss = eig_ss_class.fit_transform(temp_df);

transformed_eig_ss.eig0 = [int(x) for x in transformed_eig_ss.eig0.values]
transformed_eig_ss.head()

eig0
psn timestamp                       
34  2015-12-05 00:00:00.000000     0
    2015-12-05 00:10:00.000004     0
    2015-12-05 00:19:59.999998     0
    2015-12-05 00:30:00.000003     0
    2015-12-05 00:39:59.999997     0

In [167]:
type(clean_data)

numpy.ndarray

In [168]:
clean_df = pd.DataFrame(data= clean_data, columns = clean_data_cols)
clean_df['psn'] = data.psn.values
clean_df['timestamp'] = data['timestamp'].values

In [169]:
#uses jareds refactored code for power jumps
power_df = data
power_df.set_index(['psn', 'timestamp'], inplace=True)
power_df= power_df[['perf_pow']]
power_class = PowerStepSize(power_col='perf_pow')
transformed_pow = power_class.fit_transform(power_df);
transformed_pow.perf_pow = [int(x) for x in transformed_pow.perf_pow.values]
transformed_pow.head()

psn  timestamp                 
34   2015-12-05 00:00:00.000000    False
     2015-12-05 00:10:00.000004    False
     2015-12-05 00:19:59.999998    False
     2015-12-05 00:30:00.000003    False
     2015-12-05 00:39:59.999997    False
     2015-12-05 00:50:00.000002    False
     2015-12-05 00:59:59.999996    False
     2015-12-05 01:10:00.000001    False
     2015-12-05 01:19:59.999995    False
     2015-12-05 01:30:00.000000    False
     2015-12-05 01:40:00.000004    False
     2015-12-05 01:49:59.999998    False
     2015-12-05 02:00:00.000003    False
     2015-12-05 02:09:59.999997    False
     2015-12-05 02:20:00.000002    False
     2015-12-05 02:29:59.999996    False
     2015-12-05 02:40:00.000001    False
     2015-12-05 02:49:59.999995    False
     2015-12-05 03:00:00.000000    False
     2015-12-05 03:10:00.000004    False
     2015-12-05 03:19:59.999998    False
     2015-12-05 03:30:00.000003    False
     2015-12-05 03:39:59.999997    False
     2015-12-05 03:50:00.

psn  timestamp                 
40   2017-08-12 04:30:00.000000    False
     2017-08-12 04:40:00.000004    False
     2017-08-12 04:49:59.999998    False
     2017-08-12 05:00:00.000003    False
     2017-08-12 05:09:59.999997    False
     2017-08-12 05:20:00.000002    False
     2017-08-12 05:29:59.999996    False
     2017-08-12 05:40:00.000001    False
     2017-08-12 05:49:59.999995    False
     2017-08-12 06:00:00.000000    False
     2017-08-12 06:10:00.000004    False
     2017-08-12 06:19:59.999998    False
     2017-08-12 06:30:00.000003    False
     2017-08-12 06:39:59.999997    False
     2017-08-12 06:50:00.000002    False
     2017-08-12 06:59:59.999996    False
     2017-08-12 07:10:00.000001    False
     2017-08-12 07:19:59.999995    False
     2017-08-12 07:30:00.000000    False
     2017-08-12 07:40:00.000004    False
     2017-08-12 07:49:59.999998    False
     2017-08-12 08:00:00.000003    False
     2017-08-12 08:09:59.999997    False
     2017-08-12 08:20:00.

Name: perf_pow, Length: 10941, dtype: bool
psn  timestamp                 
48   2015-12-05 00:00:00.000000    False
     2015-12-05 00:10:00.000004    False
     2015-12-05 00:19:59.999998    False
     2015-12-05 00:30:00.000003    False
     2015-12-05 00:39:59.999997    False
     2015-12-05 00:50:00.000002    False
     2015-12-05 00:59:59.999996    False
     2015-12-05 01:10:00.000001    False
     2015-12-05 01:19:59.999995    False
     2015-12-05 01:30:00.000000    False
     2015-12-05 01:40:00.000004    False
     2015-12-05 01:49:59.999998    False
     2015-12-05 02:00:00.000003    False
     2015-12-05 02:09:59.999997    False
     2015-12-05 02:20:00.000002    False
     2015-12-05 02:29:59.999996    False
     2015-12-05 02:40:00.000001    False
     2015-12-05 02:49:59.999995    False
     2015-12-05 03:00:00.000000    False
     2015-12-05 03:10:00.000004    False
     2015-12-05 03:19:59.999998    False
     2015-12-05 03:30:00.000003    False
     2015-12-05 03:39:5

psn  timestamp                 
53   2017-02-21 22:10:00.000001    False
     2017-02-21 22:19:59.999995    False
     2017-02-21 22:30:00.000000    False
     2017-02-21 22:40:00.000004    False
     2017-02-21 22:49:59.999998    False
     2017-02-21 23:00:00.000003    False
     2017-02-21 23:09:59.999997    False
     2017-02-21 23:20:00.000002    False
     2017-02-21 23:29:59.999996    False
     2017-02-21 23:40:00.000001    False
     2017-02-21 23:49:59.999995    False
     2017-02-22 00:00:00.000000    False
     2017-02-22 00:10:00.000004    False
     2017-02-22 00:19:59.999998    False
     2017-02-22 00:30:00.000003    False
     2017-02-22 00:39:59.999997    False
     2017-02-22 00:50:00.000002    False
     2017-02-22 00:59:59.999996    False
     2017-02-22 01:10:00.000001    False
     2017-02-22 01:19:59.999995    False
     2017-02-22 01:30:00.000000    False
     2017-02-22 01:40:00.000004    False
     2017-02-22 01:49:59.999998    False
     2017-02-22 02:00:00.

psn  timestamp                 
58   2016-07-27 13:10:00.000001    False
     2016-07-27 13:19:59.999995    False
     2016-07-27 13:30:00.000000    False
     2016-07-27 13:40:00.000004    False
     2016-07-27 13:49:59.999998    False
     2016-07-27 14:00:00.000003    False
     2016-07-27 14:09:59.999997    False
     2016-07-27 14:20:00.000002    False
     2016-07-27 14:29:59.999996    False
     2016-07-27 14:40:00.000001    False
     2016-07-27 14:49:59.999995    False
     2016-07-27 15:00:00.000000    False
     2016-07-27 15:10:00.000004    False
     2016-07-27 15:19:59.999998    False
     2016-07-27 15:30:00.000003    False
     2016-07-27 15:39:59.999997    False
     2016-07-27 15:50:00.000002    False
     2016-07-27 15:59:59.999996    False
     2016-07-27 16:10:00.000001    False
     2016-07-27 16:19:59.999995    False
     2016-07-27 16:30:00.000000    False
     2016-07-27 16:40:00.000004    False
     2016-07-27 16:49:59.999998    False
     2016-07-27 17:00:00.

psn  timestamp                 
62   2017-03-14 14:00:00.000003    False
     2017-03-14 14:09:59.999997    False
     2017-03-14 14:20:00.000002    False
     2017-03-14 14:29:59.999996    False
     2017-03-14 14:40:00.000001    False
     2017-03-14 14:49:59.999995    False
     2017-03-14 15:00:00.000000    False
     2017-03-14 15:10:00.000004    False
     2017-03-14 15:19:59.999998    False
     2017-03-14 15:30:00.000003    False
     2017-03-14 15:39:59.999997    False
     2017-03-14 15:50:00.000002    False
     2017-03-14 15:59:59.999996    False
     2017-03-14 16:10:00.000001    False
     2017-03-14 16:19:59.999995    False
     2017-03-14 16:30:00.000000    False
     2017-03-14 16:40:00.000004    False
     2017-03-14 16:49:59.999998    False
     2017-03-14 17:00:00.000003    False
     2017-03-14 17:09:59.999997    False
     2017-03-14 17:20:00.000002    False
     2017-03-14 17:29:59.999996    False
     2017-03-14 17:40:00.000001    False
     2017-03-14 17:49:59.

psn  timestamp                 
66   2016-06-29 10:49:59.999998    False
     2016-06-29 11:00:00.000003    False
     2016-06-29 11:09:59.999997    False
     2016-06-29 11:20:00.000002    False
     2016-06-29 11:29:59.999996    False
     2016-06-29 11:40:00.000001    False
     2016-06-29 11:49:59.999995    False
     2016-06-29 12:00:00.000000    False
     2016-06-29 12:10:00.000004    False
     2016-06-29 12:19:59.999998    False
     2016-06-29 12:30:00.000003    False
     2016-06-29 12:39:59.999997    False
     2016-06-29 12:50:00.000002    False
     2016-06-29 12:59:59.999996    False
     2016-06-29 13:10:00.000001    False
     2016-06-29 13:19:59.999995    False
     2016-06-29 13:30:00.000000    False
     2016-06-29 13:40:00.000004    False
     2016-06-29 13:49:59.999998    False
     2016-06-29 14:00:00.000003    False
     2016-06-29 14:09:59.999997    False
     2016-06-29 14:20:00.000002    False
     2016-06-29 14:29:59.999996    False
     2016-06-29 14:40:00.

psn  timestamp                 
71   2016-06-07 12:19:59.999998    False
     2016-06-07 12:30:00.000003    False
     2016-06-07 12:39:59.999997    False
     2016-06-07 12:50:00.000002    False
     2016-06-07 12:59:59.999996    False
     2016-06-07 13:10:00.000001    False
     2016-06-07 13:19:59.999995    False
     2016-06-07 13:30:00.000000    False
     2016-06-07 13:40:00.000004    False
     2016-06-07 13:49:59.999998    False
     2016-06-07 14:00:00.000003    False
     2016-06-07 14:09:59.999997    False
     2016-06-07 14:20:00.000002    False
     2016-06-07 14:29:59.999996    False
     2016-06-07 14:40:00.000001     True
     2016-06-07 14:49:59.999995    False
     2016-06-07 15:00:00.000000    False
     2016-06-07 15:10:00.000004    False
     2016-06-07 15:19:59.999998    False
     2016-06-07 15:30:00.000003    False
     2016-06-07 15:39:59.999997    False
     2016-06-07 15:50:00.000002    False
     2016-06-07 15:59:59.999996    False
     2016-06-07 16:10:00.

perf_pow
psn timestamp                           
34  2015-12-05 00:00:00.000000         0
    2015-12-05 00:10:00.000004         0
    2015-12-05 00:19:59.999998         0
    2015-12-05 00:30:00.000003         0
    2015-12-05 00:39:59.999997         0

In [170]:
labels_df = transformed_eig_ss.merge(transformed_pow, left_index=True, right_index=True)

In [171]:
labels_df.head()

eig0  perf_pow
psn timestamp                                 
34  2015-12-05 00:00:00.000000     0         0
    2015-12-05 00:10:00.000004     0         0
    2015-12-05 00:19:59.999998     0         0
    2015-12-05 00:30:00.000003     0         0
    2015-12-05 00:39:59.999997     0         0

In [172]:
labels_df[(labels_df.perf_pow ==1)|(labels_df.eig0 ==1)];

In [173]:
labels_df[(labels_df.perf_pow ==1)].shape

(7634, 2)

In [174]:
labels_df[(labels_df.eig0 ==1)].shape[0]/ len(labels_df)

0.004354414516296133

In [175]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [176]:
accuracy_score = metrics.classification_report(labels_df.perf_pow.values, labels_df.eig0.values, 
                                               digits = 5,target_names=['normal','transient'],sample_weight=None)
print(accuracy_score)

             precision    recall  f1-score   support

     normal    0.99598   0.99639   0.99619   1594648
  transient    0.17543   0.16034   0.16755      7634

avg / total    0.99207   0.99241   0.99224   1602282



In [177]:
kinks = pd.read_csv('model2_20min_kinkfinder.csv')
kinks.set_index(['psn', 'timestamp'], inplace=True)
kinks.head()

kink_finder_label
psn timestamp                             
34  2015-12-09 09:00:00                1.0
    2015-12-09 09:10:00                1.0
    2016-01-17 01:40:00                1.0
    2016-01-17 01:50:00                1.0
    2016-02-03 05:40:00                1.0

In [194]:
#jareds power
pow_df = pd.read_csv('model2_powerstepsize.csv')

In [196]:
pow_df.set_index(['psn', 'timestamp'], inplace=True)
pow_df.head()

perf_pow
psn timestamp                    
34  2015-12-05 00:00:00     False
    2015-12-05 00:10:00     False
    2015-12-05 00:20:00     False
    2015-12-05 00:30:00     False
    2015-12-05 00:40:00     False

In [197]:
kink_val = pow_df.join(kinks, how='left')
kink_val.head()

perf_pow  kink_finder_label
psn timestamp                                       
34  2015-12-05 00:00:00     False                NaN
    2015-12-05 00:10:00     False                NaN
    2015-12-05 00:20:00     False                NaN
    2015-12-05 00:30:00     False                NaN
    2015-12-05 00:40:00     False                NaN

In [198]:
kink_val.kink_finder_label.fillna(0, inplace=True)
kink_val.head()

perf_pow  kink_finder_label
psn timestamp                                       
34  2015-12-05 00:00:00     False                0.0
    2015-12-05 00:10:00     False                0.0
    2015-12-05 00:20:00     False                0.0
    2015-12-05 00:30:00     False                0.0
    2015-12-05 00:40:00     False                0.0

In [199]:
accuracy_score2 = metrics.classification_report(kink_val.perf_pow.values, kink_val.kink_finder_label.values, 
                                               digits = 5,target_names=['normal','transient'],sample_weight=None)
print(accuracy_score2)

             precision    recall  f1-score   support

     normal    0.99620   0.99734   0.99677   1594648
  transient    0.27025   0.20540   0.23340      7634

avg / total    0.99274   0.99357   0.99314   1602282



In [179]:
transformed_pow = transformed_pow.reset_index()
transformed_pow.timestamp = transformed_pow.timestamp.dt.round('10min')

In [180]:
transformed_pow.set_index(['psn', 'timestamp'], inplace=True)

In [181]:
transformed_pow.head()

perf_pow
psn timestamp                    
34  2015-12-05 00:00:00         0
    2015-12-05 00:10:00         0
    2015-12-05 00:20:00         0
    2015-12-05 00:30:00         0
    2015-12-05 00:40:00         0

In [184]:
kink_labels_df = kinks.join(transformed_pow, how='right')
kink_labels_df.head()

kink_finder_label  perf_pow
psn timestamp                                       
34  2015-12-05 00:00:00                NaN         0
    2015-12-05 00:10:00                NaN         0
    2015-12-05 00:20:00                NaN         0
    2015-12-05 00:30:00                NaN         0
    2015-12-05 00:40:00                NaN         0

In [ ]:
kink_labels_df[kin]

In [186]:
kink_labels_df.fillna(0, inplace=True)
kink_labels_df.head()

kink_finder_label  perf_pow
psn timestamp                                       
34  2015-12-05 00:00:00                0.0         0
    2015-12-05 00:10:00                0.0         0
    2015-12-05 00:20:00                0.0         0
    2015-12-05 00:30:00                0.0         0
    2015-12-05 00:40:00                0.0         0

In [193]:
labels_dive = labels_df[(labels_df.perf_pow ==1)|(labels_df.eig0 ==1)]
labels_dive.xs(37,level='psn', drop_level=False);

In [192]:
kink_labels_df[(kink_labels_df.kink_finder_label==1.0)]

,,kink_finder_label,perf_pow
psn,timestamp,,


In [190]:
kink_labels_df[(kink_labels_df.kink_finder_label==)|(kink_labels_df.perf_pow==1)]

kink_finder_label  perf_pow
psn timestamp                                       
34  2015-12-09 09:10:00                0.0         1
    2015-12-09 09:20:00                0.0         1
    2015-12-18 18:40:00                0.0         1
    2015-12-18 19:50:00                0.0         1
    2016-01-17 01:50:00                0.0         1
    2016-02-03 05:40:00                0.0         1
    2016-02-03 05:50:00                0.0         1
    2016-02-05 18:50:00                0.0         1
    2016-02-05 19:40:00                0.0         1
    2016-02-25 14:00:00                0.0         1
    2016-02-25 14:30:00                0.0         1
    2016-02-25 14:40:00                0.0         1
    2016-02-25 15:20:00                0.0         1
    2016-02-25 16:30:00                0.0         1
    2016-02-26 09:50:00                0.0         1
    2016-02-26 11:40:00                0.0         1
    2016-02-26 11:50:00                0.0         1
    2016-02-26 15:20:00                0.0         1
    2016-02-26 15:30:00                0.0         1
    2016-03-11 18:50:00                0.0         1
    2016-03-11 19:00:00                0.0         1
    2016-03-14 08:40:00                0.0         1
    2016-03-14 08:50:00                0.0         1
    2016-03-16 16:10:00                0.0         1
    2016-03-16 16:20:00                0.0         1
    2016-04-07 15:30:00                0.0         1
    2016-04-07 15:40:00                0.0         1
    2016-04-07 23:50:00                0.0         1
    2016-04-08 00:00:00                0.0         1
    2016-04-12 16:10:00                0.0         1
...                                    ...       ...
72  2017-09-05 09:40:00                0.0         1
    2017-09-05 09:50:00                0.0         1
    2017-09-05 10:30:00                0.0         1
    2017-09-05 10:50:00                0.0         1
    2017-09-05 11:10:00                0.0         1
    2017-09-05 11:50:00                0.0         1
    2017-09-05 12:00:00                0.0         1
    2017-09-12 11:40:00                0.0         1
    2017-10-02 09:10:00                0.0         1
    2017-10-02 15:20:00                0.0         1
    2017-10-03 18:50:00                0.0         1
    2017-10-03 19:00:00                0.0         1
    2017-10-03 19:10:00                0.0         1
    2017-10-27 19:20:00                0.0         1
    2017-10-27 19:30:00                0.0         1
    2017-10-27 19:40:00                0.0         1
    2017-11-09 11:00:00                0.0         1
    2017-11-09 13:40:00                0.0         1
    2017-11-09 15:30:00                0.0         1
    2017-11-09 16:00:00                0.0         1
    2017-11-09 16:20:00                0.0         1
    2017-11-10 10:30:00                0.0         1
    2017-11-10 11:10:00                0.0         1
    2017-11-20 09:30:00                0.0         1
    2017-11-20 16:30:00                0.0         1
    2017-11-20 16:50:00                0.0         1
    2017-11-22 10:00:00                0.0         1
    2017-11-22 10:30:00                0.0         1
    2017-11-22 10:50:00                0.0         1
    2017-11-22 11:00:00                0.0         1

[7634 rows x 2 columns]

In [188]:
accuracy_score2 = metrics.classification_report(kink_labels_df.perf_pow.values, kink_labels_df.kink_finder_label.values, 
                                               digits = 5,target_names=['normal','transient'],sample_weight=None)
print(accuracy_score2)

             precision    recall  f1-score   support

     normal    0.99524   1.00000   0.99761   1594648
  transient    0.00000   0.00000   0.00000      7634

avg / total    0.99049   0.99524   0.99286   1602282



/Users/jillianjarrett/anaconda2/envs/py36envir/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
